#First: explore the data

In [1]:
import pandas as pd

In [2]:
# Load ALL CSV data files
product_data = pd.read_csv('PRODUCTS.csv')
transaction_data = pd.read_csv('TRANSACTION.csv')
user_data = pd.read_csv('USER.csv')

In [3]:
# View the first rows
print(product_data.head())
print(transaction_data.head())
print(user_data.head())

          CATEGORY_1              CATEGORY_2                   CATEGORY_3  \
0  Health & Wellness           Sexual Health  Conductivity Gels & Lotions   
1             Snacks           Puffed Snacks         Cheese Curls & Puffs   
2  Health & Wellness               Hair Care        Hair Care Accessories   
3  Health & Wellness               Oral Care                   Toothpaste   
4  Health & Wellness  Medicines & Treatments               Essential Oils   

  CATEGORY_4                                       MANUFACTURER  \
0        NaN                                                NaN   
1        NaN                                                NaN   
2        NaN                           PLACEHOLDER MANUFACTURER   
3        NaN                                  COLGATE-PALMOLIVE   
4        NaN  MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...   

             BRAND       BARCODE  
0              NaN  7.964944e+11  
1              NaN  2.327801e+10  
2          ELECSOP  4.618178e

In [4]:
# Checking for missing values with the CSV files

print("Missing values within Products:\n", product_data.isnull().sum(),"\n")
print("Missing values within Transactions: \n", transaction_data.isnull().sum(),"\n")
print("Missing values within User: \n", user_data.isnull().sum())



Missing values within Products:
 CATEGORY_1          79
CATEGORY_2         976
CATEGORY_3       40571
CATEGORY_4      522201
MANUFACTURER    152004
BRAND           152002
BARCODE           2670
dtype: int64 

Missing values within Transactions: 
 RECEIPT_ID           0
PURCHASE_DATE        0
SCAN_DATE            0
STORE_NAME           0
USER_ID              0
BARCODE           5762
FINAL_QUANTITY       0
FINAL_SALE           0
dtype: int64 

Missing values within User: 
 ID                  0
CREATED_DATE        0
BIRTH_DATE       3675
STATE            4812
LANGUAGE        30508
GENDER           5892
dtype: int64


In [5]:
# Checking for duplicate rows in each csv file

print("Duplicate rows in Products: \n", product_data.duplicated().sum(),"\n")
print("Duplicate rows in Transactions: \n", transaction_data.duplicated().sum(),"\n")
print("Duplicate rows in Users: \n ", user_data.duplicated().sum())

Duplicate rows in Products: 
 101 

Duplicate rows in Transactions: 
 171 

Duplicate rows in Users: 
  0


In [6]:
# looking at summary statistics
print(product_data.info(),"\n")

print(transaction_data.info(),"\n")

print(user_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567285 entries, 0 to 567284
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   CATEGORY_1    567206 non-null  object 
 1   CATEGORY_2    566309 non-null  object 
 2   CATEGORY_3    526714 non-null  object 
 3   CATEGORY_4    45084 non-null   object 
 4   MANUFACTURER  415281 non-null  object 
 5   BRAND         415283 non-null  object 
 6   BARCODE       564615 non-null  float64
dtypes: float64(1), object(6)
memory usage: 30.3+ MB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   RECEIPT_ID      50000 non-null  object 
 1   PURCHASE_DATE   50000 non-null  object 
 2   SCAN_DATE       50000 non-null  object 
 3   STORE_NAME      50000 non-null  object 
 4   USER_ID         50000 non-null  object 
 5   BARCOD

#Second: provide SQL queries

##What are the top 5 brands by receipts scanned among users 21 and over?


In [7]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect('data.db')

# Loaded our data
user_data = pd.read_csv('USER.csv')
user_data.to_sql('users', conn, if_exists='replace', index=False)

transaction_data = pd.read_csv('TRANSACTION.csv')
transaction_data.to_sql('brands', conn, if_exists='replace', index=False)

product_data = pd.read_csv('PRODUCTS.csv')
product_data.to_sql('receipts', conn, if_exists='replace', index=False)

# Upload Our Query
top_brand_query = """
WITH Age_Users AS (
    SELECT ID AS USER_ID
    FROM users
    WHERE strftime('%Y', 'now') - strftime('%Y', BIRTH_DATE) >= 21
),
Scanned_Receipts AS (
    SELECT T.BARCODE, COUNT(T.RECEIPT_ID) AS Total_Scans
    FROM brands T
    JOIN Age_Users AEU ON T.USER_ID = AEU.USER_ID
    GROUP BY T.BARCODE
)
SELECT R.BRAND, SR.Total_Scans
FROM Scanned_Receipts SR
JOIN receipts R ON SR.BARCODE = R.BARCODE
ORDER BY SR.Total_Scans DESC
LIMIT 5;
"""

# Run the query
result = pd.read_sql(top_brand_query, conn)
print("Top 5 Brands by Receipts Scanned Among Users 21 and Over:")
print(result)


conn.close()


Top 5 Brands by Receipts Scanned Among Users 21 and Over:
         BRAND  Total_Scans
0  NERDS CANDY            4
1      TRIDENT            4
2     TOSTITOS            2
3        SUAVE            2
4         None            2


##What are the top 5 brands by sales among users that have had their account for at least six months?

In [8]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect('data.db')

# Loaded our data
user_data = pd.read_csv('USER.csv')
user_data.to_sql('users', conn, if_exists='replace', index=False)

transaction_data = pd.read_csv('TRANSACTION.csv')
transaction_data.to_sql('brands', conn, if_exists='replace', index=False)

product_data = pd.read_csv('PRODUCTS.csv')
product_data.to_sql('receipts', conn, if_exists='replace', index=False)

# Query
account_query = """
WITH Users_Account AS (
    SELECT ID AS USER_ID
    FROM users
    WHERE strftime('%Y', 'now') - strftime('%Y', CREATED_DATE) >= 1
    AND strftime('%m', 'now') - strftime('%m', CREATED_DATE) >= 6
),
Sales AS (
    SELECT T.BARCODE, SUM(T.FINAL_SALE) AS Total_Sales
    FROM brands T
    JOIN Users_Account AEU ON T.USER_ID = AEU.USER_ID
    GROUP BY T.BARCODE
)
SELECT R.BRAND, SUM(S.Total_Sales) AS Total_Sales
FROM Sales S
JOIN receipts R ON S.BARCODE = R.BARCODE
GROUP BY R.BRAND
ORDER BY Total_Sales DESC
LIMIT 5;
"""


result = pd.read_sql(account_query, conn)
print("Top 5 Brands by Sales Among Users Who Have Had Their Account for At Least Six Months:")
print(result)


conn.close()


Top 5 Brands by Sales Among Users Who Have Had Their Account for At Least Six Months:
         BRAND  Total_Sales
0          CVS        72.00
1  GREAT VALUE        12.08
2        CREST         8.97
3       ORAL-B         8.00
4      NABISCO         7.76


#Who are Fetch’s power users?

In [9]:
import sqlite3
import pandas as pd

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('data.db')

# Load data from CSV files into tables
user_data = pd.read_csv('USER.csv')
user_data.to_sql('users', conn, if_exists='replace', index=False)

transaction_data = pd.read_csv('TRANSACTION.csv')
transaction_data.to_sql('brands', conn, if_exists='replace', index=False)

product_data = pd.read_csv('PRODUCTS.csv')
product_data.to_sql('receipts', conn, if_exists='replace', index=False)

# Query to identify Fetch's power users (those who spent the most)
power_users_query = """
WITH User_Spend AS (
    SELECT T.USER_ID, SUM(T.FINAL_SALE) AS Total_Spend
    FROM brands T
    GROUP BY T.USER_ID
)
SELECT U.ID, US.Total_Spend
FROM User_Spend US
JOIN users U ON US.USER_ID = U.ID
ORDER BY US.Total_Spend DESC
LIMIT 10;  -- Top 10 users by total spend
"""

# Run the query and display results
result = pd.read_sql(power_users_query, conn)
print("Fetch's Power Users (Top 10 by Total Spend):")
print(result)

# Close the database connection
conn.close()


Fetch's Power Users (Top 10 by Total Spend):
                         ID  Total_Spend
0  643059f0838dd2651fb27f50        75.99
1  62ffec490d9dbaff18c0a999        52.28
2  5f4c9055e81e6f162e3f6fa8        37.96
3  5d191765c8b1ba28e74e8463        34.96
4  6351760a3a4a3534d9393ecd        27.74
5  64dd9170516348066e7c4006        26.52
6  62c09104baa38d1a1f6c260e        20.28
7  61a58ac49c135b462ccddd1c        19.92
8  6661ed1e7c0469953bfc76c4        18.60
9  5b441360be53340f289b0795        18.32
